# Service Mesh - Environment Review

teaser: Learn about HashiCups

<details><summary><h2>Lab Architecture</b></summary>

<img src="https://raw.githubusercontent.com/hashicorp/field-workshops-consul/master/instruqt-tracks/consul-life-of-a-developer/assets/diagrams/application-architecture.png">
</details>

---

  In this assignment, take a few moments to review the lab and the overall architecture for HashiCups.
You can see the repository for the HashiCups demo application here: https://github.com/hashicorp-demoapp. <br>


You can also view the services already running in your primary Kube cluster.

> Prometheus will be deployed in a future assignment. <br>

**QUICK FIXES**

In [ ]:
chmod 600 /root/.kube/config
kubectl config set-context k8s1 --namespace=default
kubectl config set-context k8s2 --namespace=default

Check the first k8s cluster - `k8s1`. <br>

In [ ]:
kubectl config use-context k8s1
kubectl get deployments
kubectl get pods
kubectl get svc

Check the second k8s cluster - `k8s2`. <br>

In [ ]:
kubectl config use-context k8s2
kubectl get deployments
kubectl get pods
kubectl get svc

In the next few assignments you will validate the Consul deployment.

**Command Summary**

In [ ]:
kubectl --context k8s1 get deployments
kubectl --context k8s2 get deployments
kubectl --context k8s1 get pods
kubectl --context k8s2 get pods
kubectl --context k8s1 get svc
kubectl --context k8s2 get svc

# Service Mesh - Connect Your Runtimes - Part 1

teaser: Connect Stateless K8s Cluster

tabs:
- title: K8s1 - Dashboard Token
  type: code
  hostname: workstation
  path: /root/k8s1-dashboard-token.txt
- title: K8s1 - Dashboard
  type: service
  hostname: k8s1
  path: /api/v1/namespaces/kubernetes-dashboard/services/https:kubernetes-dashboard:/proxy/
  port: 8001
- title: Consul UI
  type: service
  hostname: workstation
  path: /ui/
  port: 8500
- title: Helm Config
  type: code
  hostname: workstation
  path: /root/helm
- title: Vault UI
  type: service
  hostname: k8s1
  path: /
  port: 8200
- title: K8s Deployment
  type: code
  hostname: workstation
  path: /root/deployments

---

You can see the complete guide on Kubernetes multi-cluster federation here:
- https://www.consul.io/docs/k8s/installation/multi-cluster/kubernetes <br>

Start validating Consul on `k8s1` with Helm.
> This cluster will run our stateless workloads.  <br>

In [ ]:
kubectl config use-context k8s1
helm status consul

Sample Output
```text
...
NAME: consul
LAST DEPLOYED: Fri Sep  2 16:03:34 2022
NAMESPACE: default
STATUS: deployed
REVISION: 1
...
```

Here are the commands that was used to deploy consul.
```bash
#k8s1
kubectl config use-context k8s1
helm install -f /root/helm/k8s1-consul-values.yaml consul hashicorp/consul \
  --version $CONSUL_HELM_VERSION --wait --debug
kubectl get secret consul-federation -o yaml > consul-federation-secret.yaml
```

Check the pods. <br>

In [ ]:
kubectl get pods -l app=consul

Sample Output
```text
NAME                                          READY   STATUS    RESTARTS   AGE
consul-webhook-cert-manager-cbd46f96d-82qlm   1/1     Running   0          27m
consul-server-0                               1/1     Running   0          27m
consul-controller-cc6b67fc6-6mcnw             1/1     Running   0          27m
consul-connect-injector-568686765c-fszsm      1/1     Running   0          27m
consul-connect-injector-568686765c-tpbfb      1/1     Running   0          27m
consul-client-6ccrb                           1/1     Running   0          27m
consul-sync-catalog-d948f55f8-9kb8j           1/1     Running   0          27m
consul-mesh-gateway-b956b4566-m9sh7           2/2     Running   0          27m
consul-ingress-gateway-658cb7b9f7-hdcmr       2/2     Running   0          27m
```

Check the Consul API. <br>

In [ ]:
curl localhost:8500/v1/status/leader

You can also see the Consul UI is now up. <br>

## Set up the proxy and mesh defaults for the mesh

Review the configuration. <br>

In [ ]:
printf "\n#==> Proxy defaults\n"
cat deployments/config/proxy-defaults.yml

printf "\n#==> Mesh defaults\n"
cat deployments/config/mesh.yml

Apply the config. <br>

In [ ]:
kubectl apply -f deployments/config/proxy-defaults.yml
kubectl apply -f deployments/config/mesh.yml

Sample Output
```text
proxydefaults.consul.hashicorp.com/global created
mesh.consul.hashicorp.com/mesh created
```

In the next assignment you will federate with the other K8s clusters.

**Command Summary**

kubectl config use-context k8s1
kubectl apply -f deployments/config/proxy-defaults.yml
kubectl apply -f deployments/config/mesh.yml

**Verify**

In [ ]:
#check leader
leader=$(curl -s -o /dev/null -w "%{http_code}" http://127.0.0.1:8500/v1/status/leader)
if [ "$leader" != "200" ]; then
  fail-message "Your Consul cluster is does not have a leader."
fi

#check config
if [ "$(consul config read -kind proxy-defaults -name global | jq -r '.Config.protocol')" != "http" ]; then
  fail-message "Default protocol is not http"
fi

#check mesh
if [ "$(consul config read -kind mesh -name mesh | jq -r .TransparentProxy.MeshDestinationsOnly)" != "false" ]; then
  fail-message "Mesh destination is not true"
fi

# Service Mesh - Connect Your Runtimes - Part 2

teaser: Connect Stateful K8s Cluster

---

In this assignment you will validate the steps in this guide for the secondary cluster: https://www.consul.io/docs/k8s/installation/multi-cluster/kubernetes <br>

Deploy Consul to `k8s2` cluster.

> Consul has already been pre-deployed. Now, we just check the status.

In [ ]:
kubectl config use-context k8s2
helm status consul

Sample Output
```text
NAME: consul
LAST DEPLOYED: Fri Sep  2 16:05:33 2022
NAMESPACE: default
STATUS: deployed
REVISION: 1
```

Here are the commands that was used to deploy consul.
```bash
#k8s2
kubectl config use-context k8s2
kubectl apply -f consul-federation-secret.yaml
helm install -f /root/helm/k8s2-consul-values.yaml consul hashicorp/consul \
  --version $CONSUL_HELM_VERSION --wait --debug
```

Check the pods. <br>

In [ ]:
kubectl get pods -l app=consul

```text
NAME                                          READY   STATUS    RESTARTS   AGE
consul-webhook-cert-manager-cbd46f96d-86bhc   1/1     Running   0          24m
consul-server-0                               1/1     Running   0          24m
consul-controller-7dcf5f7c55-psqg4            1/1     Running   0          24m
consul-client-lxwzl                           1/1     Running   0          24m
consul-connect-injector-686764b98d-b2wd8      1/1     Running   0          24m
consul-connect-injector-686764b98d-ztkxw      1/1     Running   0          24m
consul-mesh-gateway-8487fc9b5f-gqf2m          2/2     Running   0          24m
```

Check that the Kubernetes clusters are federated.

In [ ]:
consul members -wan

```text
Node                  Address          Status  Type    Build   Protocol  DC    Partition  Segment
consul-server-0.k8s1  10.42.0.32:8302  alive   server  1.11.4  2         k8s1  default    <all>
consul-server-0.k8s2  10.42.0.21:8302  alive   server  1.11.4  2         k8s2  default    <all>
```

In this new few assignments, you will deploy workloads to these clusters.

**Command Summary**

kubectl config use-context k8s2

**Verify**

In [ ]:
#check leader
leader=$(curl -s -o /dev/null -w "%{http_code}" http://127.0.0.1:8500/v1/status/leader?dc=k8s2)
if [ "$leader" != "200" ]; then
  fail-message "Your Consul cluster is does not have a leader."
fi

# Service Mesh - Deploy Your Application -  Stateful

teaser: Deploy Application Storage

tabs:
- title: K8s2 - Dashboard Token
  type: code
  hostname: workstation
  path: /root/k8s2-dashboard-token.txt
- title: K8s2 - Dashboard
  type: service
  hostname: k8s2
  path: /api/v1/namespaces/kubernetes-dashboard/services/https:kubernetes-dashboard:/proxy/
  port: 8001
- title: K8s Deployment
  type: code
  hostname: workstation
  path: /root/deployments/storage

---

In this assignment you will deploy the stateful storage of your application to the K8s2 cluster. <br>

Deploy the stateful storage components.

In [ ]:
kubectl config use-context k8s2
kubectl apply -f storage

Output
```text
service/payments-queue created
serviceaccount/payments-queue created
servicedefaults.consul.hashicorp.com/payments-queue created
statefulset.apps/payments-queue created
service/product-db created
serviceaccount/product-db created
servicedefaults.consul.hashicorp.com/product-db created
statefulset.apps/product-db created
serviceintentions.consul.hashicorp.com/product-db created
serviceintentions.consul.hashicorp.com/payments-queue created
```

Wait for the storage pods to be ready.

In [ ]:
kubectl wait pod --selector=app=payments-queue  --for=condition=Ready
kubectl wait pod --selector=app=product-db  --for=condition=Ready

Output
```text
pod/payments-queue-0 condition met
pod/product-db-0 condition met
```

In the next assignment you will connect workloads on the other K8s cluster to the deployed storage tier.

<b>Command Summary</b>

```shell
cd /root/deployments
kubectl config use-context k8s2
kubectl apply -f storage
kubectl wait pod --selector=app=payments-queue  --for=condition=Ready
kubectl wait pod --selector=app=product-db  --for=condition=Ready
```

**Verify**

In [ ]:
```shell
#context
kubectl config use-context k8s2

#pods
kubectl wait pod --for=condition=Ready --selector=app=payments-queue --timeout=30s
if [ $? -ne 0 ]
then
  fail-message "Payment Queue pod not deployed."
fi
kubectl wait pod --for=condition=Ready --selector=app=product-db --timeout=30s
if [ $? -ne 0 ]
then
  fail-message "Product DB pod not deployed."
fi
```

# Service Mesh - Deploy Your Application - Stateless

teaser: Deploy reactive and API components

tabs:

- title: K8s1 - Dashboard Token
  type: code
  hostname: workstation
  path: /root/k8s1-dashboard-token.txt
- title: K8s1 - Dashboard
  type: service
  hostname: k8s1
  path: /api/v1/namespaces/kubernetes-dashboard/services/https:kubernetes-dashboard:/proxy/
  port: 8001
- title: Consul UI
  type: service
  hostname: workstation
  path: /ui/k8s1
  port: 8500
- title: K8s Deployment
  type: code
  hostname: workstation
  path: /root/deployments/v1

---

In this assignment you will deploy the stateless components of your application to the K8s1 cluster. <br>

Now deploy the frontend, public API, payments API, and product API components.

In [ ]:
kubectl config use-context k8s1
kubectl apply -f v1

Output
```text
service/frontend created
serviceaccount/frontend created
configmap/nginx created
deployment.apps/frontend created
service/payments-api created
serviceaccount/payments-api created
configmap/payments-properties-conf-v1 created
deployment.apps/payments-api-v1 created
service/product-api created
serviceaccount/product-api created
configmap/conf created
deployment.apps/product-api created
service/public-api created
serviceaccount/public-api created
deployment.apps/public-api created
serviceintentions.consul.hashicorp.com/frontend created
serviceintentions.consul.hashicorp.com/public-api created
serviceintentions.consul.hashicorp.com/product-api created
serviceintentions.consul.hashicorp.com/payments-api created
```

Wait for the app tier to be ready.

In [ ]:
kubectl wait pod --for=condition=Ready --selector=app=frontend
kubectl wait pod --for=condition=Ready --selector=app=public-api
kubectl wait pod --for=condition=Ready --selector=app=product-api
kubectl wait pod --for=condition=Ready --selector=app=payments-api,version=v1 --timeout 90s

In the next assignment you will test the application, and the connectivity other K8s cluster.

**Command Summary**

```shell
cd /root/deployments

#update the deployment for the instruqt sandbox
sed -i \
  "s,http://localhost,https://k8s1-8080-$_SANDBOX_ID.env.play.instruqt.com,g" \
  /root/deployments/v1/frontend.yml

kubectl config use-context k8s1
kubectl apply -f v1
kubectl wait pod --for=condition=Ready --selector=app=frontend
kubectl wait pod --for=condition=Ready --selector=app=public-api
kubectl wait pod --for=condition=Ready --selector=app=product-api
kubectl wait pod --for=condition=Ready --selector=app=payments-api,version=v1 --timeout 90s
```

**Verify**

In [ ]:
kubectl config use-context k8s1

kubectl wait pod --for=condition=Ready --selector=app=frontend
if [ $? -ne 0 ]
then
  echo "Frontend pod not deployed."
fi
kubectl wait pod --for=condition=Ready --selector=app=public-api
if [ $? -ne 0 ]
then
  echo "Public API pod not deployed."
fi
kubectl wait pod --for=condition=Ready --selector=app=product-api
if [ $? -ne 0 ]
then
  echo "Product API pod not deployed."
fi
kubectl wait pod --for=condition=Ready --selector=app=payments-api,version=v1
if [ $? -ne 0 ]
then
  echo "Payment API V1 pod not deployed."
fi

# Traffic Management: Ingress

teaser: Bring traffic into your mesh

tabs:
- title: K8s Deployment
  type: code
  hostname: workstation
  path: /root/deployments/ingress

---

Review and apply the ingress config.

In [ ]:
kubectl config use-context k8s1
kubectl apply -f ingress/hashicups.yml

Your ingress config is now in Consul. <br>

In [ ]:
kubectl describe ingressgateway ingress-gateway
consul config read -kind ingress-gateway -name ingress-gateway | jq

In the next assignment you will configure request routing for this ingress gateway.

**Command Summary**

```shell
kubectl config use-context k8s1
kubectl apply -f ingress/hashicups.yml
```

**Verify**

In [ ]:
#check config
if [ "$(consul config read -kind ingress-gateway -name ingress-gateway  | jq -r '.Listeners[0].Services[0].Name')" != "hashicups" ]; then
  fail-message "Ingress Gateway not configured for Hashicups"
fi

# Traffic Management: Request Routing

teaser: Apply advanced routing patterns

tabs:

- title: K8s1 - Dashboard Token
  type: code
  hostname: workstation
  path: /root/k8s1-dashboard-token.txt
- title: K8s1 - Dashboard
  type: service
  hostname: k8s1
  path: /api/v1/namespaces/kubernetes-dashboard/services/https:kubernetes-dashboard:/proxy/
  port: 8001
- title: Consul UI
  type: service
  hostname: workstation
  path: /ui/k8s1
  port: 8500
- title: K8s Deployment
  type: code
  hostname: workstation
  path: /root/deployments/ingress
- title: App
  type: service
  hostname: k8s1
  path: /
  port: 8080
  new_window: true

---

Review the routes and apply the routing configuration.  <br>

In [ ]:
kubectl config use-context k8s1
kubectl apply -f ingress/service-router.yml

Reload the App tab to see the application that is now served over the ingress gateway. <br>

In [ ]:
kubectl exec deploy/consul-ingress-gateway -c ingress-gateway -- \
  wget -qO- 127.0.0.1:19000/config_dump \
  | jq '[.. |."dynamic_route_configs"? \| select(. != null)[0]]'

You can also inspect the clusters. <br>

kubectl exec deploy/consul-ingress-gateway -c ingress-gateway -- wget -qO- 127.0.0.1:19000/clusters

In the next few assignments you will test HashiCups and use the traffic management capabilities of Consul to ship a new feature in this application.

**Command Summary**

```shell
kubectl config use-context k8s1
kubectl apply -f ingress/service-router.yml
```

**Verify**

In [ ]:
#context
kubectl config use-context k8s1

#check our routes
if [ "$(consul config read -kind service-router -name hashicups  | jq -r '.Routes[0].Destination.Service')" != "public-api" ]; then
  fail-message "Public API route is not set."
fi

if [ "$(consul config read -kind service-router -name hashicups  | jq -r '.Routes[1].Destination.Service')" != "frontend" ]; then
  fail-message "Frontend route is not set."
fi

# Service Mesh - Test Your Application

teaser: Test your deployment
tabs:
- title: App
  type: service
  hostname: k8s1
  path: /
  port: 8080
  new_window: true
- title: K8s Deployment
  type: code
  hostname: workstation
  path: /root/deployments/v1

---

Your application should now be available. You can quickly test the APIs by exposing the public API as a NodePort services.

> NodePort services are uncommon in production, but suitable for dev and test. <br>

First, let's look at the differences for upstream services located in the same `k8s1` cluster, and services in a different cluster. <br>

In [ ]:
kubectl config use-context k8s1

Same cluster: Public API <br>
The address will be for a sidecar pod. <br>

In [ ]:
kubectl exec deploy/public-api -c envoy-sidecar -- \
  wget -qO- 127.0.0.1:19000/clusters | grep product

External cluster: Payments API <br>
The address will be for the local mesh gateway running in the cluster. <br>

In [ ]:
kubectl exec deploy/payments-api-v1 -c envoy-sidecar -- \
  wget -qO- 127.0.0.1:19000/clusters | grep payments-queue

The gateway will inspect the SNI headers and forward it along to the correct destination.

Inspect the SNI value for this service now. <br>

In [ ]:
kubectl exec deploy/payments-api-v1 -c envoy-sidecar -- \
  wget -qO- 127.0.0.1:19000/config_dump | jq '[.. |."dynamic_active_clusters"? | select(. != null)[2]]'

Now that you understand the internals of the cross cluster routing, let's test the application so you can make sure it works. <br>

In [ ]:
kubectl describe svc consul-ingress-gateway
ip=$(kubectl get svc consul-ingress-gateway -o json \
  | jq -r '.status.loadBalancer.ingress[0].ip')

Try the product API. <br>

In [ ]:
curl -s -v http://${ip}:8080/api \
  -H 'Accept-Encoding: gzip, deflate, br' \
  -H 'Content-Type: application/json' \
  -H 'Accept: application/json' \
  -H 'Connection: keep-alive' \
  -H 'DNT: 1' \
  --data-binary '{"query":"{\n  coffees{id,name,price}\n}"}' \
  --compressed | jq

Try the payment API. <br>

In [ ]:
curl -s -v http://${ip}:8080/api \
  -H 'Accept-Encoding: gzip, deflate, br' \
  -H 'Content-Type: application/json' \
  -H 'Accept: application/json' \
  -H 'Connection: keep-alive' \
  -H 'DNT: 1' \
  --data-binary '{"query":"mutation{ pay(details:{ name: \"nic\", type: \"mastercard\", number: \"1234123-0123123\", expiry:\"10/02\", cv2: 1231, amount: 12.23 }){id, card_plaintext, card_ciphertext, message } }"}' \
  --compressed | jq

You should have received a 200 status code from the app API.  <br>

Optionally, you can review the data directly in the storage tier.
There will be at least one payment in the queue after running the above APIs.

Switch to the other K8s cluster. <br>

In [ ]:
kubectl config use-context k8s2

Check the payment queue. <br>

In [ ]:
kubectl exec statefulset/payments-queue -- redis-cli KEYS '*'

Check the database. <br>

In [ ]:
kubectl exec statefulset/product-db -- \
  env PGPASSWORD=postgres psql -U postgres -d products \
  -c 'SELECT * FROM coffees' -a

In the next few assignments you will be introduced to more advanced traffic management patterns.

**Verify**

In [ ]:
#context
kubectl config use-context k8s1

#check the gateway
ip=$(kubectl get svc consul-ingress-gateway -o json | jq -r '.status.loadBalancer.ingress[0].ip')
if [ -z "$ip" ]; then
  fail-message "Ingress Gateway IP not set."
fi

#get the ingress gateway
kubectl describe svc consul-ingress-gateway
ip=$(kubectl get svc consul-ingress-gateway -o json | jq -r '.status.loadBalancer.ingress[0].ip')

#check product
product=$(curl -s -o /dev/null -w "%{http_code}" http://${ip}:8080/api \
-H 'Accept-Encoding: gzip, deflate, br' \
-H 'Content-Type: application/json' \
-H 'Accept: application/json' \
-H 'Connection: keep-alive' \
-H 'DNT: 1' \
--data-binary '{"query":"{\n  coffees{id,name,price}\n}"}' \
--compressed)
if [ "$product" != "200" ]; then
  fail-message "Product API did not return a 200."
fi

#check payment
payment=$(curl -s -o /dev/null -w "%{http_code}" http://${ip}:8080/api \
-H 'Accept-Encoding: gzip, deflate, br' \
-H 'Content-Type: application/json' \
-H 'Accept: application/json' \
-H 'Connection: keep-alive' \
-H 'DNT: 1' \
--data-binary '{"query":"mutation{ pay(details:{ name: \"nic\", type: \"mastercard\", number: \"1234123-0123123\", expiry:\"10/02\", cv2: 1231, amount: 12.23 }){id, card_plaintext, card_ciphertext, message } }"}' \
--compressed)
if [ "$payment" != "200" ]; then
  fail-message "Payment API did not return a 200."
fi

**Command Summary**

---
slug: service-mesh-service-discovery
id: 6g4ky4xdgapb
type: challenge
title: 'Service Mesh: Service Discovery'
teaser: Scale up & Scale Down
tabs:
- title: App
  type: service
  hostname: k8s1
  path: /
  port: 8080
  new_window: true
- title: Workstation
  type: terminal
  hostname: workstation
- title: Consul UI
  type: service
  hostname: workstation
  path: /ui
  port: 8500
- title: K8s Deployment
  type: code
  hostname: workstation
  path: /root/deployments
- title: Lab Architecture
  type: website
  url: https://htmlpreview.github.io/?https://raw.githubusercontent.com/hashicorp/field-workshops-consul/master/instruqt-tracks/consul-life-of-a-developer/assets/diagrams/diagrams.html
difficulty: basic
timelimit: 500
---
Service Discovery is a important component of Service Mesh, and K8s is one of many platforms Consul supports native service discovery.
Understanding of the low level data plane is not required, but it can be helpful to get a deeper understanding of the mesh at work!

When running on K8s, Consul can consume K8s probes to determine if an instance is healthy or not to receive traffic.
You will test this workflow below. <br>

Add add curl to the payments api pods. <br>

```
kubectl config use-context k8s1
kubectl exec deploy/payments-api-v1 -c payments-api -- apk add curl
```

First let's look at the health check on *one* of our payments API pods. It will be passing. <br>

```
kubectl exec deploy/payments-api-v1 -c payments-api -- curl -s 127.0.0.1:8080/actuator/health | jq
```

Check the downstream service view of the payments-api upstream. You will see two services, and their pod IPs.

```
kubectl exec deploy/public-api -c envoy-sidecar -- wget -qO- 127.0.0.1:19000/clusters | grep payments-api
```

We can use the chaos library on our payments service to simulate an application failure on one of our payments pods, and then look at its health check.


```
kubectl exec deploy/payments-api-v1 -c payments-api -- curl -s -X POST localhost:8080/actuator/chaosmonkey/enable
sleep 15
```

Check the health check endpoint. <br>

```
kubectl exec deploy/payments-api-v1 -c payments-api -- curl -s 127.0.0.1:8080/actuator/health | jq
```

Check the K8s probe. <br>

```
kubectl get pod --selector=app=payments-api,version=v1 -o json | jq .items[0].status.conditions
```

Check the status in Consul. <br>

```
curl -s http://127.0.0.1:8500/v1/health/checks/payments-api | jq
```

This failure will remove this instance from the data plane. You will see this reflected in Envoy's health flag.
The flag will prevent this instance from receiving traffic, and the value will be `/failed_eds_health` <br>

Check the downstream service for this flag. <br>

```
kubectl exec deploy/public-api -c envoy-sidecar -- wget -qO- 127.0.0.1:19000/clusters | grep payments-api
```

Your payments service will now fail.

```
ip=$(kubectl get svc consul-ingress-gateway -o json | jq -r '.status.loadBalancer.ingress[0].ip')
curl -s -v http://${ip}:8080/api \
-H 'Accept-Encoding: gzip, deflate, br' \
-H 'Content-Type: application/json' \
-H 'Accept: application/json' \
-H 'Connection: keep-alive' \
-H 'DNT: 1' \
--data-binary '{"query":"mutation{ pay(details:{ name: \"nic\", type: \"mastercard\", number: \"1234123-0123123\", expiry:\"10/02\", cv2: 1231, amount: 12.23 }){id, card_plaintext, card_ciphertext, message } }"}' \
--compressed | jq
```

Scale the deployment to bring a heathly instance online, and wait for the pod to start. Consul will transparently shift traffic away from the failed pod.

```
kubectl scale deployment.v1.apps/payments-api-v1 --replicas=2
sleep 60
```

Check the status in Consul. You will two sets of service checks. One of the sets will be healthy. <br>

```
curl -s http://127.0.0.1:8500/v1/health/checks/payments-api | jq
```

Now check the upstream service.

```
kubectl exec deploy/public-api -c envoy-sidecar -- wget -qO- 127.0.0.1:19000/clusters | grep payments-api | grep health_flags
```

Try your payments service again. <br>

```
ip=$(kubectl get svc consul-ingress-gateway -o json | jq -r '.status.loadBalancer.ingress[0].ip')
curl -s -v http://${ip}:8080/api \
-H 'Accept-Encoding: gzip, deflate, br' \
-H 'Content-Type: application/json' \
-H 'Accept: application/json' \
-H 'Connection: keep-alive' \
-H 'DNT: 1' \
--data-binary '{"query":"mutation{ pay(details:{ name: \"nic\", type: \"mastercard\", number: \"1234123-0123123\", expiry:\"10/02\", cv2: 1231, amount: 12.23 }){id, card_plaintext, card_ciphertext, message } }"}' \
--compressed | jq
```

You can run a graceful restart of the deployment to cycle away the bad pod. You can proceed to the next challenge during the rollout.

```
kubectl rollout restart deployment/payments-api-v1
```

You will introduce more advanced traffic management in the next few assignments.


In [ ]:
#context
kubectl config use-context k8s1

#get the ingress gateway
kubectl describe svc consul-ingress-gateway
ip=$(kubectl get svc consul-ingress-gateway -o json | jq -r '.status.loadBalancer.ingress[0].ip')

#check product
product=$(curl -s -o /dev/null -w "%{http_code}" http://${ip}:8080/api \
  -H 'Accept-Encoding: gzip, deflate, br' \
  -H 'Content-Type: application/json' \
  -H 'Accept: application/json' \
  -H 'Connection: keep-alive' \
  -H 'DNT: 1' \
  --data-binary '{"query":"{\n  coffees{id,name,price}\n}"}' \
  --compressed)
if [ "$product" != "200" ]; then
  fail-message "Product API did not return a 200."
fi

#check payments
payment=$(curl -s -o /dev/null -w "%{http_code}" http://${ip}:8080/api \
  -H 'Accept-Encoding: gzip, deflate, br' \
  -H 'Content-Type: application/json' \
  -H 'Accept: application/json' \
  -H 'Connection: keep-alive' \
  -H 'DNT: 1' \
  --data-binary '{"query":"mutation{ pay(details:{ name: \"nic\", type: \"mastercard\", number: \"1234123-0123123\", expiry:\"10/02\", cv2: 1231, amount: 12.23 }){id, card_plaintext, card_ciphertext, message } }"}' \
  --compressed)
if [ "$payment" != "200" ]; then
  fail-message "Payment API did not return a 200."
fi

**Command Summary**

```shell
#context
kubectl config use-context k8s1

#chaos
kubectl exec deploy/payments-api-v1 -c payments-api -- apk add curl
kubectl exec deploy/payments-api-v1 -c payments-api -- curl -s -X POST localhost:8080/actuator/chaosmonkey/enable
sleep 15

#scale
kubectl scale deployment.v1.apps/payments-api-v1 --replicas=2
sleep 60

#restart
kubectl rollout restart deployment/payments-api-v1
```

# Appendix

- Setup scripts - https://github.com/hashicorp/field-workshops-consul/blob/master/instruqt-tracks/consul-life-of-a-developer/track_scripts/setup-workstation